# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json 


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = pd.read_csv("../output_data/cities.csv")

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [13]:
output_data_file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


## Generate Cities List

In [14]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


626

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
cities[:8]

['airai',
 'mwinilunga',
 'klaksvik',
 'takoradi',
 'albany',
 'yicheng',
 'oksfjord',
 'punta arenas']

In [ ]:
#create empty lists
city_name = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

#weather api 
url = 'http://api.openweathermap.org/data/2.5/weather'

#print statement
print('Beginning Data Retrieval')
print('-----------------------------')

record = 1
for city in cities:
    #create parameters for api call 
    params={
        "q":city,
        "units":"imperial",
        "appid": weather_api_key
    }
    try:
        response = requests.get(url, params)
        print(response.url)
        results = response.json()
        print(json.dumps(results, indent=4, sort_keys=True))
        city_name.append(results["city"])
    except Exception:
        print("City not found. Skipping...")

Beginning Data Retrieval
-----------------------------
http://api.openweathermap.org/data/2.5/weather?q=saint+anthony&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 90
    },
    "cod": 200,
    "coord": {
        "lat": 45.02,
        "lon": -93.22
    },
    "dt": 1584632838,
    "id": 5044290,
    "main": {
        "feels_like": 34.05,
        "humidity": 93,
        "pressure": 1013,
        "temp": 38.73,
        "temp_max": 39.99,
        "temp_min": 37
    },
    "name": "Saint Anthony",
    "rain": {
        "1h": 0.25
    },
    "sys": {
        "country": "US",
        "id": 4900,
        "sunrise": 1584620211,
        "sunset": 1584663845,
        "type": 1
    },
    "timezone": -18000,
    "visibility": 2414,
    "weather": [
        {
            "description": "light rain",
            "icon": "10d",
            "id": 500,
            "main": "Rain"
        },
        {
            "description": "mist",
   

http://api.openweathermap.org/data/2.5/weather?q=turayf&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 40
    },
    "cod": 200,
    "coord": {
        "lat": 31.67,
        "lon": 38.66
    },
    "dt": 1584632682,
    "id": 101312,
    "main": {
        "feels_like": 44.82,
        "humidity": 41,
        "pressure": 1012,
        "temp": 57.2,
        "temp_max": 57.2,
        "temp_min": 57.2
    },
    "name": "Turaif",
    "sys": {
        "country": "SA",
        "id": 7432,
        "sunrise": 1584588611,
        "sunset": 1584632156,
        "type": 1
    },
    "timezone": 10800,
    "visibility": 10000,
    "weather": [
        {
            "description": "scattered clouds",
            "icon": "03n",
            "id": 802,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 250,
        "speed": 16.11
    }
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=yel

http://api.openweathermap.org/data/2.5/weather?q=opuwo&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": -18.06,
        "lon": 13.84
    },
    "dt": 1584632684,
    "id": 3354077,
    "main": {
        "feels_like": 76.73,
        "grnd_level": 870,
        "humidity": 55,
        "pressure": 1008,
        "sea_level": 1008,
        "temp": 80.51,
        "temp_max": 80.51,
        "temp_min": 80.51
    },
    "name": "Opuwo",
    "rain": {
        "3h": 2.81
    },
    "sys": {
        "country": "NA",
        "sunrise": 1584594499,
        "sunset": 1584638179
    },
    "timezone": 7200,
    "weather": [
        {
            "description": "light rain",
            "icon": "10d",
            "id": 500,
            "main": "Rain"
        }
    ],
    "wind": {
        "deg": 102,
        "speed": 14.5
    }
}
City not found. Skipping...
http://api.openweathermap.or

http://api.openweathermap.org/data/2.5/weather?q=orapa&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": -21.28,
        "lon": 25.37
    },
    "dt": 1584632686,
    "id": 933186,
    "main": {
        "feels_like": 83.59,
        "grnd_level": 898,
        "humidity": 30,
        "pressure": 1010,
        "sea_level": 1010,
        "temp": 85.73,
        "temp_max": 85.73,
        "temp_min": 85.73
    },
    "name": "Orapa",
    "sys": {
        "country": "BW",
        "sunrise": 1584591721,
        "sunset": 1584635423
    },
    "timezone": 7200,
    "weather": [
        {
            "description": "overcast clouds",
            "icon": "04d",
            "id": 804,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 90,
        "speed": 4.27
    }
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=mana&units=imp

http://api.openweathermap.org/data/2.5/weather?q=utiroa&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "cod": "404",
    "message": "city not found"
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=aykhal&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 90
    },
    "cod": 200,
    "coord": {
        "lat": 66,
        "lon": 111.5
    },
    "dt": 1584632688,
    "id": 2027296,
    "main": {
        "feels_like": -4.56,
        "humidity": 71,
        "pressure": 986,
        "temp": 6.8,
        "temp_max": 6.8,
        "temp_min": 6.8
    },
    "name": "Aykhal",
    "sys": {
        "country": "RU",
        "id": 8846,
        "sunrise": 1584657251,
        "sunset": 1584701323,
        "type": 1
    },
    "timezone": 32400,
    "visibility": 10000,
    "weather": [
        {
            "description": "overcast clouds",
            "icon": "04n",
            "id": 804,
      

http://api.openweathermap.org/data/2.5/weather?q=mezen&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 84
    },
    "cod": 200,
    "coord": {
        "lat": 65.85,
        "lon": 44.24
    },
    "dt": 1584632690,
    "id": 527321,
    "main": {
        "feels_like": 9.86,
        "grnd_level": 996,
        "humidity": 93,
        "pressure": 1000,
        "sea_level": 1000,
        "temp": 19.85,
        "temp_max": 19.85,
        "temp_min": 19.85
    },
    "name": "Mezen'",
    "sys": {
        "country": "RU",
        "sunrise": 1584587183,
        "sunset": 1584630906
    },
    "timezone": 10800,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04n",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 16,
        "speed": 8.57
    }
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=mys+shmidta&units

http://api.openweathermap.org/data/2.5/weather?q=luza&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 100
    },
    "cod": 200,
    "coord": {
        "lat": 60.63,
        "lon": 47.25
    },
    "dt": 1584632692,
    "id": 533143,
    "main": {
        "feels_like": 24.1,
        "grnd_level": 978,
        "humidity": 95,
        "pressure": 998,
        "sea_level": 998,
        "temp": 31.55,
        "temp_max": 31.55,
        "temp_min": 31.55
    },
    "name": "Luza",
    "snow": {
        "3h": 0.13
    },
    "sys": {
        "country": "RU",
        "sunrise": 1584586497,
        "sunset": 1584630147
    },
    "timezone": 10800,
    "weather": [
        {
            "description": "light snow",
            "icon": "13n",
            "id": 600,
            "main": "Snow"
        }
    ],
    "wind": {
        "deg": 310,
        "speed": 6.44
    }
}
City not found. Skipping...
http://api.openweathermap.org/data

http://api.openweathermap.org/data/2.5/weather?q=sinnamary&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 53
    },
    "cod": 200,
    "coord": {
        "lat": 5.38,
        "lon": -52.95
    },
    "dt": 1584632694,
    "id": 3380290,
    "main": {
        "feels_like": 80.78,
        "grnd_level": 1014,
        "humidity": 76,
        "pressure": 1014,
        "sea_level": 1014,
        "temp": 80.69,
        "temp_max": 80.69,
        "temp_min": 80.69
    },
    "name": "Sinnamary",
    "rain": {
        "3h": 0.38
    },
    "sys": {
        "country": "GF",
        "sunrise": 1584610572,
        "sunset": 1584654158
    },
    "timezone": -10800,
    "weather": [
        {
            "description": "light rain",
            "icon": "10d",
            "id": 500,
            "main": "Rain"
        }
    ],
    "wind": {
        "deg": 57,
        "speed": 15.61
    }
}
City not found. Skipping...
http://api.openweat

http://api.openweathermap.org/data/2.5/weather?q=odweyne&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 80
    },
    "cod": 200,
    "coord": {
        "lat": 9.41,
        "lon": 45.06
    },
    "dt": 1584632695,
    "id": 53372,
    "main": {
        "feels_like": 77.79,
        "grnd_level": 884,
        "humidity": 23,
        "pressure": 1010,
        "sea_level": 1010,
        "temp": 86.09,
        "temp_max": 86.09,
        "temp_min": 86.09
    },
    "name": "Oodweyne",
    "sys": {
        "country": "SO",
        "sunrise": 1584587062,
        "sunset": 1584630633
    },
    "timezone": 10800,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04n",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 179,
        "speed": 12.24
    }
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=faya&units=i

http://api.openweathermap.org/data/2.5/weather?q=chimoio&units=imperial&appid=69054b289ead344e4cf8c465c34738df
{
    "base": "stations",
    "clouds": {
        "all": 40
    },
    "cod": 200,
    "coord": {
        "lat": -19.12,
        "lon": 33.48
    },
    "dt": 1584632698,
    "id": 1049261,
    "main": {
        "feels_like": 80.82,
        "humidity": 69,
        "pressure": 1018,
        "temp": 78.8,
        "temp_max": 78.8,
        "temp_min": 78.8
    },
    "name": "Chimoio",
    "sys": {
        "country": "MZ",
        "id": 2199,
        "sunrise": 1584589781,
        "sunset": 1584633471,
        "type": 1
    },
    "timezone": 7200,
    "visibility": 10000,
    "weather": [
        {
            "description": "scattered clouds",
            "icon": "03d",
            "id": 802,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 120,
        "speed": 8.03
    }
}
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?q=m

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [6]:
# OPTIONAL: Create a function to create Linear Regression plots


In [7]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression